In [17]:
import os
from glob import glob
from collections import Counter

import pandas as pd
import numpy as np

Read the classification splits

In [4]:
survey = 'Gaia'
folds = glob('../../../../CV_c/01_CreateFolds/{}/Folds/*'.format(survey))
folds.sort()

test_path = folds[-1]
folds = folds[:-1] # Ignore test set
folds

['../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_1',
 '../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_2',
 '../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_3',
 '../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_4',
 '../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_5',
 '../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_6',
 '../../../../CV_c/01_CreateFolds/Gaia/Folds/Fold_7']

Read the data

In [5]:
path = '/home/iebecker/Desktop/iebecker_stuff/Disks/Data/Paper_2/Prepare_dataset/Gaia/V6/Dataset_Gaia_Phys_V6.dat'
df = pd.read_csv(path).set_index('ID')
df.head()

,Path,N,N_b,N_r,Class,T_eff,e_T_eff,E_T_eff,Lum,e_Lum,...,E_Rad,logg,e_logg,E_logg,Mass,e_Mass,E_Mass,rho,e_rho,E_rho
ID,,,,,,,,,,,,,,,,,,,,,
3985923473972534400,/home/Data/Databases/GAIA/Consolidate_Gaia/dat...,8,4,4,DSCT_SXPHE,-1.0000,-1.000,-1.0000,-1.00000,-1.0,...,-1.0,-1.0000,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0000,-1.0,-1.0
3986570197263160320,/home/Data/Databases/GAIA/Consolidate_Gaia/dat...,10,5,5,RRAB,6481.0000,-1.000,-1.0000,-1.00000,-1.0,...,-1.0,-1.0000,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0000,-1.0,-1.0
3986754189367115264,/home/Data/Databases/GAIA/Consolidate_Gaia/dat...,12,6,6,RRAB,7381.3335,7173.793,7558.6665,2.83599,-1.0,...,-1.0,4.5185,-1.0,-1.0,1.55,-1.0,-1.0,1.0603,-1.0,-1.0
3987237630885709312,/home/Data/Databases/GAIA/Consolidate_Gaia/dat...,12,7,5,RRAB,7011.0000,-1.000,-1.0000,-1.00000,-1.0,...,-1.0,-1.0000,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0000,-1.0,-1.0
3987697089307190016,/home/Data/Databases/GAIA/Consolidate_Gaia/dat...,8,4,4,RRAB,7191.5000,6808.000,8884.0000,2.30248,-1.0,...,-1.0,4.6352,-1.0,-1.0,1.58,-1.0,-1.0,1.5719,-1.0,-1.0


In [8]:
path_folds = './Folds'
if not os.path.exists(path_folds):
    os.mkdir(path_folds)

new_root = '/home/iebecker/Desktop/iebecker_stuff/Disks/'
test = pd.read_csv(test_path,usecols=['ID']).set_index('ID')
df_test = pd.merge(test, df,left_index=True, right_index=True).reset_index()
df_test.Path = df_test.Path.str.replace("/home/",new_root ) # Fix path to work here

path_test = os.path.join(path_folds, 'test.csv')
df_test.to_csv(path_test, index=False, index_label=False)

for n,fold in enumerate(folds):
    # Read classifier folds
    train = pd.read_csv(os.path.join(fold, 'train.csv'), usecols=['ID']).set_index('ID')
    val = pd.read_csv(os.path.join(fold, 'val.csv'), usecols=['ID']).set_index('ID')

    # Join with new physical params
    df_train = pd.merge(train, df,left_index=True, right_index=True).reset_index()
    df_val = pd.merge(val, df,left_index=True, right_index=True).reset_index()
    df_train.Path = df_train.Path.str.replace("/home/",new_root )
    df_val.Path = df_val.Path.str.replace("/home/",new_root )

    
    path_folds_ = os.path.join(path_folds, 'Fold_'+str(n+1))
    if not os.path.exists(path_folds_):
        os.mkdir(path_folds_)
        
    path_train = os.path.join(path_folds_, 'train.csv')
    path_val = os.path.join(path_folds_, 'val.csv')
    
    df_train.to_csv(path_train, index=False, index_label=False)
    df_val.to_csv(path_val, index=False, index_label=False)

In [9]:
data_train = pd.read_csv(path_train)
data_val = pd.read_csv(path_val)
data_test = pd.read_csv(path_test)
data_train.head()

,ID,Path,N,N_b,N_r,Class,T_eff,e_T_eff,E_T_eff,Lum,...,E_Rad,logg,e_logg,E_logg,Mass,e_Mass,E_Mass,rho,e_rho,E_rho
0,6863003013366410752,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,27,14,13,RRAB,6336.75,5950.0,6911.0,13.41960,...,-1.00000,3.6941,-1.0,-1.0,1.39,-1.0,-1.0,0.0649,-1.0,-1.0
1,6815011353741855104,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,24,12,12,RRAB,6627.67,6302.0,7013.0,4.56033,...,-1.00000,4.1192,-1.0,-1.0,1.34,-1.0,-1.0,0.2872,-1.0,-1.0
2,4667949299731626496,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,26,14,12,RRAB,6146.00,-1.0,-1.0,-1.00000,...,-1.00000,-1.0000,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0000,-1.0,-1.0
3,1218995148346916480,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,53,26,27,MIRA_SR,-1.00,-1.0,-1.0,-1.00000,...,110.59825,-1.0000,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0000,-1.0,-1.0
4,4294499904783165696,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,36,18,18,MIRA_SR,-1.00,-1.0,-1.0,-1.00000,...,-1.00000,-1.0000,-1.0,-1.0,-1.00,-1.0,-1.0,-1.0000,-1.0,-1.0


In [14]:
def class_proportion(df):
    dict_ = Counter(df.Class)
    dict_ = {key:np.round(value/df.shape[0], 3) for key, value in  zip(dict_.keys(), dict_.values())}
    return dict_

In [18]:
class_proportion(data_train)

{'RRAB': 0.372,
 'MIRA_SR': 0.372,
 'RRC': 0.16,
 'CEP': 0.054,
 'DSCT_SXPHE': 0.032,
 'T2CEP': 0.011}

In [19]:
class_proportion(data_test)

{'RRC': 0.16,
 'MIRA_SR': 0.372,
 'RRAB': 0.372,
 'DSCT_SXPHE': 0.032,
 'T2CEP': 0.011,
 'CEP': 0.054}

In [20]:
class_proportion(data_val)

{'MIRA_SR': 0.372,
 'CEP': 0.054,
 'RRAB': 0.372,
 'RRC': 0.16,
 'DSCT_SXPHE': 0.032,
 'T2CEP': 0.011}